<a href="https://colab.research.google.com/github/w731xiluo/SD_modle_trainnin_0/blob/main/stable_diffusion_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
###这是注释
!apt-get -y install -qq aria2
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install -q --pre triton

!git clone -b v1.6 https://github.com/camenduru/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone -b v1.6 https://github.com/camenduru/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone -b v1.6 https://github.com/camenduru/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -b v1.6 https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v1.6 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone -b v1.6 https://github.com/camenduru/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone -b v1.6 https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
%cd /content/stable-diffusion-webui
!git reset --hard

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd14/resolve/main/sd-v1-4.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-4.ckpt

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i '$a fastapi==0.90.0' requirements_versions.txt
!sed -i -e 's/default_enabled=False/default_enabled=True/' /content/stable-diffusion-webui/webui.py

!python launch.py --share --xformers --enable-insecure-extension-access --theme dark --cloudflared

In [ ]:
#查看是什么GPU
!nvidia-smi
#挂载谷歌硬盘
from google.colab import drive
drive.mount('/content/drive/')
!echo "google硬盘挂载完成."

In [ ]:
#安装aria2
!apt -qq install liblz4-tool aria2

#从Akegarasu的git仓库下载Lora训练模型
#################################################
#不保证资源长期有效
#若资源失效自行寻找可用git
!git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts
!echo "git的lora库克隆完成."
!cd lora-scripts && git pull && git submodule update --init --recursive
!echo "更新完成."

#安装其他依赖
%cd /content/lora-scripts/sd-scripts/
!pip -q install --upgrade -r requirements.txt

#安装xformers 0.0.16版本
!pip install xformers==0.0.16

#安装Triton
!pip -q install triton==2.0.0.dev20221202

#按照lion
!pip -q install --upgrade lion-pytorch

#从谷歌硬盘中拷贝你之前上传的训练材料
!mkdir -p /content/lora-scripts/train/aki/
!cp -r /content/drive/MyDrive/Lora/input/* /content/lora-scripts/train/aki/
!echo "copy训练材料完成."

#拷贝正则化图片
!mkdir -p /content/lora-scripts/train/reg/
!cp -r /content/drive/MyDrive/Lora/reg/* /content/lora-scripts/train/reg/
!echo "copy正则化图片完成."

%cd /content/lora-scripts